In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
data_path = '../Data/Max, Samantha, Maria data.xlsx'
xls = pd.ExcelFile(data_path)
df_max = pd.read_excel(xls, sheet_name = 'Max', parse_dates = [3])
df_mar = pd.read_excel(xls, sheet_name = 'Maria', parse_dates = [3])
df_sam = pd.read_excel(xls, sheet_name = 'Samantha', parse_dates = [3])
df_all = [df_max, df_mar, df_sam]

C:\Users\Joseph\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:211: UserWarning: Cell D2858 is marked as a date but the serial value 6684137.0 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


In [3]:
# Max df lacks coupon column; dropping
df_mar.drop(columns = 'coupon', inplace = True)
df_sam.drop(columns = 'Coupon (#)', inplace = True)

In [4]:
# Standarize column names
column_names = ['ID', 'Session', 'ReceiptNum', 'ReceiptDate', 'Item', 'Item2',
                 'Uncertain', 'Unknown', 'Quantity', 'Hit', 'Miss', 'Category', 'Comment']
for df in df_all:
    df.columns = column_names

In [16]:
# Fill NaN values
# when receipt number is null, assume all items came from a single basket that session
for df in df_all:
    df['Quantity'].fillna(value = 1, inplace = True)
    df['Item2'].fillna(value = '', inplace = True)
    df['Comment'].fillna(value = '', inplace = True)
    df.loc[pd.isna(df['ReceiptNum']), 'ReceiptNum'] = 1

In [6]:
# Assign data types after cleaning typos
df_sam.loc[df_sam.ReceiptNum == datetime.datetime(1900, 1, 1, 0, 0), 'ReceiptNum'] = 1
df_sam.loc[df_sam['Quantity'] == '??', 'Quantity'] = 1

string_columns = ['Item', 'Item2', 'Category', 'Comment']
for df in df_all:
    df[['ID', 'Session', 'ReceiptNum', 'Quantity']] = df[['ID', 'Session', 'ReceiptNum', 'Quantity']].astype(pd.Int64Dtype())
    df['ReceiptDate'] = pd.to_datetime(df['ReceiptDate'], errors = 'coerce')
    df[string_columns] = df[string_columns].astype(str)
    # clean strings
    for col in string_columns:
        df[col] = df[col].str.lower()

In [7]:
# Drop rows with essential NaN values
# Drop 'Unknown' rows
# Drop rows where 'Comments' contains "duplicate" or "repeat"
for df in df_all:
    df.dropna(subset = ['ID', 'Session', 'Item'], inplace = True)
    df.drop(pd.notnull(df['Unknown']).index)
    df.drop(df['Comment'].str.contains(r'duplicate|repeat').index)

In [17]:
for df in df_all:
    print(df.info())
    display(df.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2159 entries, 0 to 2158
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           2159 non-null   Int64         
 1   Session      2159 non-null   Int64         
 2   ReceiptNum   2159 non-null   Int64         
 3   ReceiptDate  1718 non-null   datetime64[ns]
 4   Item         2159 non-null   object        
 5   Item2        2159 non-null   object        
 6   Uncertain    37 non-null     object        
 7   Unknown      87 non-null     object        
 8   Quantity     2159 non-null   Int64         
 9   Hit          0 non-null      float64       
 10  Miss         0 non-null      float64       
 11  Category     2159 non-null   object        
 12  Comment      2159 non-null   object        
dtypes: Int64(4), datetime64[ns](1), float64(2), object(6)
memory usage: 309.1+ KB
None


,ID,Session,ReceiptNum,ReceiptDate,Item,Item2,Uncertain,Unknown,Quantity,Hit,Miss,Category,Comment
0,129,1,1,2020-06-19,soda (grapefruit),,NaN,NaN,2,NaN,NaN,drink,
1,129,1,1,2020-06-19,dried fruit,,NaN,NaN,1,NaN,NaN,fruit,
2,129,1,1,2020-06-19,wafers (lemon),,NaN,NaN,1,NaN,NaN,snack,
3,129,1,1,2020-06-19,wafers (vanilla),,NaN,NaN,1,NaN,NaN,snack,
4,129,1,1,2020-06-19,wafers (coconut),,NaN,NaN,1,NaN,NaN,snack,


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4324 entries, 0 to 4323
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           4324 non-null   Int64         
 1   Session      4324 non-null   Int64         
 2   ReceiptNum   4324 non-null   Int64         
 3   ReceiptDate  2849 non-null   datetime64[ns]
 4   Item         4324 non-null   object        
 5   Item2        4324 non-null   object        
 6   Uncertain    52 non-null     object        
 7   Unknown      95 non-null     object        
 8   Quantity     4324 non-null   Int64         
 9   Hit          0 non-null      float64       
 10  Miss         0 non-null      float64       
 11  Category     4324 non-null   object        
 12  Comment      4324 non-null   object        
dtypes: Int64(4), datetime64[ns](1), float64(2), object(6)
memory usage: 618.9+ KB
None


,ID,Session,ReceiptNum,ReceiptDate,Item,Item2,Uncertain,Unknown,Quantity,Hit,Miss,Category,Comment
0,154,1,1,2020-08-18,watermelon,,NaN,NaN,1,NaN,NaN,fruit,
1,154,1,2,2020-08-21,strawberries,,NaN,NaN,1,NaN,NaN,fruit,
2,154,1,2,2020-08-21,gold bell peppers,,NaN,NaN,1,NaN,NaN,vegetable,
3,154,1,2,2020-08-21,green bell peppers,,NaN,NaN,1,NaN,NaN,vegetable,
4,154,1,2,2020-08-21,blueberries,,NaN,NaN,1,NaN,NaN,fruit,


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3138 entries, 0 to 3137
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           3138 non-null   Int64         
 1   Session      3138 non-null   Int64         
 2   ReceiptNum   3138 non-null   Int64         
 3   ReceiptDate  2509 non-null   datetime64[ns]
 4   Item         3138 non-null   object        
 5   Item2        3138 non-null   object        
 6   Uncertain    111 non-null    object        
 7   Unknown      82 non-null     object        
 8   Quantity     3138 non-null   Int64         
 9   Hit          27 non-null     float64       
 10  Miss         0 non-null      float64       
 11  Category     3138 non-null   object        
 12  Comment      3138 non-null   object        
dtypes: Int64(4), datetime64[ns](1), float64(2), object(6)
memory usage: 420.0+ KB
None


,ID,Session,ReceiptNum,ReceiptDate,Item,Item2,Uncertain,Unknown,Quantity,Hit,Miss,Category,Comment
0,131,1,1,2020-07-21,strawberries,,NaN,NaN,1,NaN,NaN,fruit,
1,131,1,1,2020-07-21,apple cider vinegar,,NaN,NaN,4,NaN,NaN,condiment,
2,131,1,1,2020-07-21,cheddar cheese (sticks),,NaN,NaN,1,NaN,NaN,dairy,
3,131,1,2,2020-08-02,candy bar (baby ruth),,NaN,NaN,3,NaN,NaN,candy,
4,131,1,2,2020-08-02,flavored water,,NaN,NaN,1,NaN,NaN,drink,
